In [12]:
import pandas as pd
from sqlalchemy import create_engine
import re

server = 'JERAL_BENITES'
database = 'picks'
username = 'sa'
password = 'zaperoko1'

connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)

sql_query = """
SELECT 
    match.app, couponb.name cupon, categoryb.name categoria, match.bet apuesta,
    match.day dia, match.time hora, match.rate tasa, match.status estado, teamaway.name equipoVisita, teamhome.name  equipoLocal,league.name liga
FROM [dbo].[match_bet] AS match
LEFT JOIN [dbo].[team_bet] AS teamaway
       ON match.id_away_team = teamaway.id
LEFT JOIN [dbo].[team_bet] AS teamhome
       ON match.id_home_team = teamhome.id
LEFT JOIN [dbo].[league_bet] AS league
       ON match.id_league = league.id
LEFT JOIN [dbo].[category_bet] AS categoryb
       ON match.id_category = categoryb.id
LEFT JOIN [dbo].[coupon_bet] AS couponb
       ON categoryb.id_coupon = couponb.id
"""

data = pd.read_sql(sql_query, engine)

data.head()

def es_formato_fecha(texto):
    return re.match(r'\d{2} \d{2} \d{4}', texto)

In [13]:
data['liga'] = data['liga'].str.upper()
data['liga'] = data.apply(lambda row: row['categoria'] if es_formato_fecha(row['liga']) else row['liga'], axis=1)
data['liga'] = data['liga'].apply(lambda x: re.sub(r'\bEUORPE\b', 'EUROPE', x))
data['liga'] = data['liga'].apply(lambda x: re.sub(r'\bEURO \b', 'EUROPE', x))
data['liga'] = data['liga'].apply(lambda x: re.sub(r'\b W\b', ' WOMEN', x))
data['liga'] = data['liga'].apply(lambda x: re.sub(r'\bNETHERLAND \b', 'NETHERLANDS', x))
data['liga'] = data['liga'].apply(lambda x: re.sub(r'\bLA LIG\b', 'LA LIGA', x))
data['liga'] = data['liga'].apply(lambda x: re.sub(r'\b A \b', '', x))
data['liga'] = data['liga'].apply(lambda x: re.sub(r'\b A\b', '', x))
data['liga'] = data['liga'].apply(lambda x: re.sub(r'\bARGENTINAA\b', 'ARGENTINA', x))

data['liga'] = data['liga'].replace(
    {
        'AFRCICA': 'AFRICA',
        'BRZIL': 'BRAZIL',"->": " ",
        "BRZIL": "BRAZIL",
        "ENGLAND LEAGUE ONE": "ENGLAND LEAGUE",
        "CZECH MOL CUP": "CZECH REPUBLIC CUP",
        "CZECH REP. CUP": "CZECH REPUBLIC CUP",
        "CZECH REP CUP": "CZECH REPUBLIC CUP",
        "CHEZCH CHEZCH LEAGUE": "CHEZCH CHEZCH",
        "DIVSION": "DIVISION",
        "EGYPT PREMIER LEAGUE": "EGYPT LEAGUE",
        "ENGLAND EFL CUP": "ENGLAND CUP",
        "POLAND EKSTRAKALSA": "POLAND EKSTRAKLASA",
        "PORTO LIGA": "PORTUGAL LIGA",
        "WORLD CUP BASEKTBALL": "WORLD CUP BASKETBALL",
        "AFIRCA CUP": "AFRICA CUP",
        "AFRICA CUP OF NATIONS": "AFRICA CUP",
        "AFRICA NATIONS CUP": "AFRICA CUP",
        "ALGERIA LIGUE-1": "ALGERIA LEAGUE",
        "ARGENTINA PREMIER LEAGUE": "ARGENTINA LEAGUE",
        "ARGENTINA PREMIER": "ARGENTINA LEAGUE",
        "ARGENTINA PRIMERA DIVISION": "ARGENTINA LEAGUE",
        "ARGENTINA PRIMERA NACIONAL": "ARGENTINA LEAGUE",
        "ARGENTINA PRIMERIA LEAGUE": "ARGENTINA LEAGUE",
        "ARGENTINA A LEAGUE": "ARGENTINA LEAGUE",
        "AUSTRIA A": "AUSTRIA 1",
        "AUSTRIA 1.LEAGUE": "AUSTRIA 1",
        "AZERIAN PREMIER LEAGUE": "AZERIAN PREMIER",
        "BAE 1.LEAGUE": "BAE ARABIA LEAGUE",
        "BAE ARABIA LAEGUE": "BAE ARABIA LEAGUE",
        "BELGIUM 1.LEAGUE A": "BELGIUM 1.LEAGUE",
        "BOLIVIA PREMIER": "BOLIVIA LEAGUE",
        "LAEGUE": "LEAGUE",
        "LEAUGE": "LEAGUE",
        "LEGA": "LEAGUE",
        "LEGA": "LEAGUE",
        "LEAAGUE": "LEAGUE",
        "CHALLENGER(POSTP.)": "CHALLENGER",
        "CHALLENGER(POST.)": "CHALLENGER",
        "CHALLENGER(POST )": "CHALLENGER",
        "COLOMBIA LEAGUE": "COLOMBIA PRIMERA",
        "COLOMBIA PRIMERA A": "COLOMBIA PRIMERA",
        "CONCAF": "CONCACAF",
        "DEMARK 1.LEAGUE": "DENMARK 1.DIVISION",
        "DENMARK 1.LEAGUE": "DENMARK 1.DIVISION",
        "DENMARK SUPER LEAGUE": "DENMARK SUPER LIGA",
        "DENMARK SUPERLIGA": "DENMARK SUPER LIGA",
        "ENGLAND PREMIER LEAGUE": "ENGLAND PREMIER",
        "EUOPE": "EUROPE",
        "FINLAND YKONEN": "FINLAND YKKONEN",
        "AZEDAGAN": "AZADEGAN",
        "AZEDEGAN": "AZADEGAN",
        "BOTALO": "BOTOLA",
        "EEERSTE": "EERSTE",
        "EESTE": "EERSTE",
        "EREDIVISE": "EREDIVISIE",
        "EREDVSSIE": "EREDIVISIE",
        "POALND": "POLAND",
        "PREMIERSIHP": "PREMIERSHIP",
        "SOTUH": "SOUTH",
        "SPIAN": "SPAIN",
        "LA LIGA(POSTPONED)": "LA LIGA",
        "SUPERATTAN": "SUPERETTAN",
        "SWITZERLAND SUPER LEAGUE": "SWITZERLAND LEAGUE",
        "PROFFESIONAL": "PROFESSIONAL",
        "WTA(POSTP.)": "WTA",
        "GREEK CUP": "GREECE CUP",
        "TURKISH CUP": "TURKEY CUP"
    }
)

data.to_excel('datos_exportados.xlsx', index=False, engine='openpyxl')